Misc derivations of frame3d FEM elements and vefirication that results are consistent with welib.FEM.frame3d

In [ ]:
# Sympy
import sympy as sp
from sympy import symbols, Symbol, simplify, Function, expand_trig
from sympy import cos, sin
from sympy import integrate
from sympy import python, latex
from sympy import Matrix
from sympy import DiracDelta
from sympy import I, exp, pi, oo
from sympy import laplace_transform, inverse_laplace_transform
from sympy import fourier_transform, inverse_fourier_transform
from sympy.physics.vector import init_vprinting, vlatex
init_vprinting(use_latex='mathjax', pretty_print=False)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import matplotlib.pyplot as plt
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib inline
# --- local packages
# --- Main variables
tau, omega = symbols('tau, omega', real=True, positive=True, imaginary=False)
tau.assumptions0

In [ ]:
# --- Helper functions
def KXXint(SX,limInf=0):
    return integrate( SX * cos(omega * tau) , (omega, limInf, oo)  ) 

def KXXintComplex(SXX):
    return integrate( SXX * exp( I * omega * tau) , (omega, -oo, oo)  ) 

def SXint(KXX):
    return  integrate(2/pi * KXX * cos(omega * tau) , (tau, 0, oo)  ) 

def SXintComplex(KXX):
    # NOTE: return SX not SXX!
    return  integrate(1/(pi) * KXX * exp(-I * omega* tau) , (tau, -oo, oo)  ) 

## Constant autocovariance

In [ ]:
tau, omega = symbols('tau, omega', imaginary=False,  real=True)  # NOTE: positive=True, makes DiracDelta fail
sigma = symbols('sigma', real=True, positive=True, imaginary=False)
KXX= sigma**2 
SX= 2* sigma**2 * DiracDelta(omega)  # SX=2SXX
print('Theory:')
KXX
SX
print('Integration:')
KXX_ = KXXint(SX)
KXX_
SX_ =  SXint(KXX) # Faiul to retrieve Dirac
#SX2_ = SXintComplex(KXX)
#SX2_.simplify()
SX_.simplify()


In [ ]:
sp.Integral(DiracDelta(omega), (omega,-oo, oo))

# Constant spectrum (White noise)

In [ ]:
tau, omega = symbols('tau, omega', imaginary=False,  real=True)  # NOTE: positive=True, makes DiracDelta fail
S0 = symbols('S_0', real=True, positive=True, imaginary=False)
KXX= 2 * pi*S0 * DiracDelta(tau)
SX= 2*S0    # SX=2SXX
print('Theory:')
KXX
SX
print('Integration:')
KXX_ = KXXint(SX)
KXX_
SX_ =  SXint(KXX) # Faiul to retrieve Dirac
#SX2_ = SXintComplex(KXX)
#SX2_.simplify()
SX_.simplify()


# Exponential Spectrum

In [ ]:
tau, omega = symbols('tau, omega', imaginary=False,  real=True, positive=True)
sigma, lambd = symbols('sigma, lambda', real=True, positive=True, imaginary=False)
#sigma.assumptions0
SX  = sigma**2 * lambd * exp(- lambd * omega)
KXX = sigma**2 /( 1 + tau**2/lambd**2)

print('Theory:')
SX
KXX
print('Integration:')
KXX_ = KXXint(SX)
SX_ =  SXint(KXX)
SX2_ = SXintComplex(KXX)
SX2_.simplify()
SX_.simplify()
KXX_


In [ ]:
integrate(SX, (omega, 0, oo))

# Harmonic process

In [ ]:
sigma, omega0 = symbols('sigma, omega_0', real=True, positive=True, imaginary=False)
#sigma.assumptions0
KXX = sigma**2 * cos(omega0*tau)
SX  = sigma**2 * (DiracDelta(omega-omega0))

print('Theory:')
KXX
SX
print('Integration:')
KXX_ = KXXint(SX)   # No problem integrating the Dirac
KXX_
SX_ =  SXint(KXX) # Does not converge, needs to be handled analytically
SX2_ = SXintComplex(KXX) # Does not converge
SX2_.simplify()
SX_.simplify()


# Band-limited white noise

In [ ]:
sigma, B, omega0 = symbols('sigma, B, omega_0', real=True, positive=True, imaginary=False)
KXX = sigma**2/(B*tau) * (sin((omega0+B/2)*tau ) - sin((omega0-B/2)*tau))
SX1 = sp.Piecewise( ( sigma**2/B , (omega>omega0-B/2) & (omega<omega0+B/2)   ), ( 0, True)  )
SX2 = (sigma**2/B * (sp.Heaviside(omega-(omega0-B/2) ) - sp.Heaviside(omega-(omega0+B/2)) )).rewrite(sp.Piecewise)
print('Theory:')
SX1
SX2
KXX

In [ ]:
sp.Integral(SX1, (omega, -B/2, oo)).doit().simplify()
sp.Integral(SX2, (omega, -B/2, oo)).rewrite(sp.Piecewise).doit().simplify()

In [ ]:
print('Integration to get KXX:')
KXX_ = KXXint(SX1, limInf=-B/2)
KXX_
KXX_.simplify()

In [ ]:
print('Integration to get SX:')
#SX_ =  SXint(KXX)  # Does not work
#SX2_ = SXintComplex(KXX)  # gives a result but useless
#SX2_.simplify()
#SX_.simplify()

# Rational auto-spectrum

In [ ]:
tau, omega = symbols('tau, omega', imaginary=False)  # NOTE: positive=True, makes DiracDelta fail
omega0, a = symbols('omega_0, a',  positive=True, imaginary=False)
zeta = symbols('zeta',  positive=True, imaginary=False)
k = symbols('k',  positive=True, imaginary=False)
KXX= exp(-a*abs(tau) ) * cos(omega0*tau)
KXX= pi* S0 / (2*zeta*omega0**3)* exp(-zeta*omega0*abs(tau) ) * ( cos(omega0*k *tau) + zeta/k * sin(omega0*k *tau))

SXX = S0 /( ( (I*omega)**2 + 2*zeta* omega0 *omega *I + omega0**2   ) *( (-I*omega)**2 - 2*zeta* omega0 *omega *I + omega0**2   ) )
SX=2*SXX

print('Theory:')
KXX
SX
SXX
SXX=SXX.simplify()
SX = SX.simplify()
SX
SXX

In [ ]:
print('Integration of SX:')
#KXX_ = KXXint(SX)
KXX_ = KXXintComplex(SX)
KXX_

In [ ]:
print('Integration of KXX:')
#SX_ =  SXint(KXX) 
#SX_.simplify()
#SX2_ = SXintComplex(KXX)
SX2_ = integrate(1/(pi) * KXX * exp(-I * omega* tau) , (tau, -oo, oo)  , conds='none') 
SX2_ = SX2_.simplify()
MySubs=[(k**2, 1-zeta**2)]
SX2_.subs(MySubs).simplify()

In [ ]:

SX2_.subs(MySubs).expand().simplify()

In [ ]:
KXX1_ = integrate( SXX * exp(I * omega* tau) , (omega, -oo, oo) ) 
KXX1_

In [ ]:
KXX1_ = integrate( 2*SXX * cos(omega* tau) , (omega, 0, oo) ) 
KXX1_

In [ ]:
#KXX2_ = KXXint(SX)
#KXX2_